#MNist com Multi-Layer Perceptron

Neste experimento foram exploradas configurações de hiper-parâmetros do algoritmo Multi-Layer Perceptron, variando a função de ativação (*activation*): entre Tangente Hiperbólica e Relu, o *solver* que é o otimizador do algoritmo entre o *sgd* (Gradiente Descendente Estocástico) e o *adam* (Gradiente Descendente Estocástico otimizado), por fim quantidade de neurônios nas camadas escondidas testando uma unica camada com 100 neurônios, depois são duas camadas com 10 neurônios cada, em seguida duas camadas com 15 e 15, e para finalizar duas camadas com 20 neurônios em cada.

Para garantir uma reproducibilidade entre este experimento e os demais, o conjunto de dados foi separado em 10 pastas com a técnica de Cross Validation utilizando o estado randômico 42.

A combinação dos hiper-parâmetros foi feita utilizando Grid Search, que cruza todas as opções dos dicionários e responde qual é a melhor configuração encontrada. Estas parametrizações são utilizadas para criar um modelo final, que é serializado em um objeto Pickle.

In [1]:
from sklearn.datasets import fetch_openml

import joblib
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.neural_network import MLPClassifier

import pandas as pd
import numpy as np

import time

In [2]:
mnist = fetch_openml('mnist_784')

In [3]:
X = mnist["data"]
y = mnist["target"]

print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


In [4]:
param_grid = [{'activation': ['tanh', 'relu'],
               'solver':['sgd', 'adam'],
               'hidden_layer_sizes': [(100),(10,10),(15,15),(20,20)] }]

kfolds = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

In [5]:
X_treino, y_treino, X_teste, y_teste = [],[],[],[]

for idx_treino, idx_teste in kfolds.split(X, y):
  X_treino.append(idx_treino)
  y_treino.append(idx_treino)
  X_teste.append(idx_teste)
  y_teste.append(idx_teste)

In [ ]:
mlp = MLPClassifier()

inicio = time.time()
grid_search_mlp = GridSearchCV(mlp, param_grid, cv=kfolds, verbose=3)
resultado_modelo = grid_search_mlp.fit(X,y)
termino = time.time()
print("--- %s segundos para treinar o modelo ---" % (termino - inicio))

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV 1/10] END activation=tanh, hidden_layer_sizes=100, solver=sgd;, score=0.960 total time= 9.5min


In [ ]:
print("Melhores Parâmetros")
print(grid_search_mlp.best_params_)
print("**************")
print("Melhores Estimadores")
print(grid_search_mlp.best_estimator_)
print("**************")
print("Melhores Pontuações")
print(grid_search_mlp.best_score_)
print("**************")
print(grid_search_mlp.best_index_)
print("**************")

In [ ]:
joblib.dump(grid_search_mlp, "modelo_mlp_mnist.pkl")